In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import manifold
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt

from time import time

In [2]:
df_train = pd.read_csv('train_genelevel_90%.tsv', sep='\t',index_col=0)
#df.set_index(df["Unnamed: 0"], inplace=True)
df_train.shape

(55169, 407)

In [3]:
df_test = pd.read_csv('test_genelevel_10%.tsv', sep='\t',index_col=0)
#df.set_index(df["Unnamed: 0"], inplace=True)
df_test.shape

(55169, 46)

# Lasso Regularizer

In [4]:
#transpose the dataframe --> make genes as dementions 
dfn = df_train.T
dfn['mgs_level'] = 0
dfn.reset_index(inplace = True)
list = dfn['index'].apply(lambda x:x[-1]).tolist()
#list 
dfn['mgs_level'] = list
level = dfn.pop('mgs_level')
dfn.insert(1,'mgs_level',level)
dfn.head()

,index,mgs_level,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,...,ENSG00000283686,ENSG00000283688,ENSG00000283689,ENSG00000283690,ENSG00000283691,ENSG00000283693,ENSG00000283695,ENSG00000283696,ENSG00000283698,ENSG00000283699
0,205_2,2,202.0,261.0,194.0,25.0,428.00,169.0,709.0,557.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0,0.0
1,251_1,1,243.0,369.0,188.0,15.0,91.06,118.0,396.0,484.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,474_3,3,230.0,355.0,234.0,14.0,83.00,92.0,424.0,502.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,178_4,4,270.0,375.0,236.0,14.0,184.00,87.0,442.0,1020.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
4,313_1,1,232.0,317.0,385.0,14.0,167.78,94.0,403.0,747.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [5]:
## separate control vs patient
control = dfn[dfn.mgs_level == '1']
patient = dfn[dfn.mgs_level != '1']

In [6]:
## 0 if stage is 1, 1 if stage is 1,2,3
control["binary"] = 0
patient["binary"] = 1

/Users/minjunp/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/minjunp/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
## merge two datasets vertically and remove mgs level
new_df = control.append(patient, ignore_index=True)
new_df = new_df.iloc[:,2:]
new_df.head()

,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,ENSG00000001460,ENSG00000001461,...,ENSG00000283688,ENSG00000283689,ENSG00000283690,ENSG00000283691,ENSG00000283693,ENSG00000283695,ENSG00000283696,ENSG00000283698,ENSG00000283699,binary
0,243.0,369.0,188.0,15.0,91.06,118.0,396.0,484.0,533.0,1511.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0
1,232.0,317.0,385.0,14.0,167.78,94.0,403.0,747.0,305.0,1352.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
2,174.0,254.0,144.0,10.0,105.76,103.0,352.0,391.0,304.0,912.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,381.0,484.0,270.0,14.0,112.16,142.0,538.0,995.0,609.0,2163.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0
4,379.0,431.0,251.0,17.0,260.68,151.0,797.0,794.0,493.0,1814.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0


In [8]:
## print shape
new_df.shape

(407, 55170)

In [9]:
## split into X, Y variables
X_train = new_df.iloc[:,:-1]
y_train = new_df.iloc[:,-1]
X_train.shape, y_train.shape

((407, 55169), (407,))

In [10]:
import seaborn as sns
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from sklearn import datasets, linear_model

scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [11]:
sel_ = SelectFromModel(LogisticRegression(penalty='l1')) ##penalty is lasso regression
sel_.fit(scaler.transform(X_train), y_train)

/Users/minjunp/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


SelectFromModel(estimator=LogisticRegression(C=1.0, class_weight=None,
                                             dual=False, fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='warn',
                                             n_jobs=None, penalty='l1',
                                             random_state=None, solver='warn',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [12]:
sel_.get_support()

array([False, False, False, ..., False, False, False])

In [13]:
## Make a list of with the selected features.
selected_feat = X_train.columns[(sel_.get_support())]
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format(
      np.sum(sel_.estimator_.coef_ == 0)))

total features: 55169
selected features: 266
features with coefficients shrank to zero: 54903


In [14]:
selected_feat

Index(['ENSG00000007908', 'ENSG00000078579', 'ENSG00000101251',
       'ENSG00000110660', 'ENSG00000120075', 'ENSG00000125695',
       'ENSG00000125975', 'ENSG00000134398', 'ENSG00000134817',
       'ENSG00000136573',
       ...
       'ENSG00000278908', 'ENSG00000278992', 'ENSG00000279450',
       'ENSG00000279698', 'ENSG00000279773', 'ENSG00000280070',
       'ENSG00000280189', 'ENSG00000280850', 'ENSG00000283154',
       'ENSG00000283578'],
      dtype='object', length=266)

In [15]:
## Removing the features from training
## 55169 patients, 275 important features
X_train_selected = sel_.transform(X_train)
X_train_selected = np.transpose(X_train_selected)
X_train_selected.shape
#sample = X_train_selected[1:100]
#ylabel = sig_feats[1:100]
#type(ylabel)
#ylabel = ylabel.values.tolist()

(266, 407)

In [16]:
## Identifying the removed features
removed_feats = X_train.columns[(sel_.estimator_.coef_ == 0).ravel().tolist()]
removed_feats

Index(['ENSG00000000419', 'ENSG00000000457', 'ENSG00000000460',
       'ENSG00000000938', 'ENSG00000000971', 'ENSG00000001036',
       'ENSG00000001084', 'ENSG00000001167', 'ENSG00000001460',
       'ENSG00000001461',
       ...
       'ENSG00000283686', 'ENSG00000283688', 'ENSG00000283689',
       'ENSG00000283690', 'ENSG00000283691', 'ENSG00000283693',
       'ENSG00000283695', 'ENSG00000283696', 'ENSG00000283698',
       'ENSG00000283699'],
      dtype='object', length=54903)

In [17]:
X_train_selected
df = pd.DataFrame(data=X_train_selected, index=selected_feat)
df.T

,ENSG00000007908,ENSG00000078579,ENSG00000101251,ENSG00000110660,ENSG00000120075,ENSG00000125695,ENSG00000125975,ENSG00000134398,ENSG00000134817,ENSG00000136573,...,ENSG00000278908,ENSG00000278992,ENSG00000279450,ENSG00000279698,ENSG00000279773,ENSG00000280070,ENSG00000280189,ENSG00000280850,ENSG00000283154,ENSG00000283578
0,5.0,1.0,0.0,431.0,0.0,0.0,0.0,1.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,281.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,96.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,497.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,3.0,3.0,492.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,0.0,2.0,1.0,285.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
403,2.0,0.0,3.0,234.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
404,1.0,2.0,0.0,369.0,0.0,0.0,0.0,1.0,4.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
405,0.0,4.0,0.0,130.0,0.0,0.0,0.0,0.0,26.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
lasso_selected_feature = selected_feat

# Ridge Regularizer

In [19]:
sel_ = SelectFromModel(LogisticRegression(penalty='l2')) ##penalty is ridge regression
sel_.fit(scaler.transform(X_train), y_train)

/Users/minjunp/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


SelectFromModel(estimator=LogisticRegression(C=1.0, class_weight=None,
                                             dual=False, fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='warn',
                                             n_jobs=None, penalty='l2',
                                             random_state=None, solver='warn',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [20]:
sel_.get_support()

array([False, False, False, ...,  True, False, False])

In [21]:
## Make a list of with the selected features.
selected_feat = X_train.columns[(sel_.get_support())]
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format(
      np.sum(sel_.estimator_.coef_ == 0)))

total features: 55169
selected features: 20765
features with coefficients shrank to zero: 12063


In [22]:
ridge_selected_feature = selected_feat

# Elastic Net regularizer

In [23]:
#sel_ = SelectFromModel(LogisticRegression(penalty='elasticnet')) ##penalty is ridge regression
#sel_.fit(scaler.transform(X_train), y_train)

In [24]:
#sel_.get_support()

# Low variance filtering

In [25]:
from sklearn.feature_selection import VarianceThreshold

In [26]:
sel = VarianceThreshold(threshold=1)
res = sel.fit_transform(X_train)
print(X_train.shape)
print(res.shape)

(407, 55169)
(407, 23945)


In [27]:
def variance_threshold_selector(data, threshold=0.5):
    selector = VarianceThreshold(threshold)
    selector.fit(data)
    return data[data.columns[selector.get_support(indices=True)]]

In [28]:
df_variance = variance_threshold_selector(X_train, threshold=1)

In [29]:
variance_selected_feature = df_variance.columns ## Feature selector that removes all low-variance features.

# Compare Lasso, Rigid, Low variance filtering

In [30]:
variance_selected_feature ## 23945 features
lasso_selected_feature ## 266 features 
ridge_selected_feature ## 20765 features

Index(['ENSG00000000938', 'ENSG00000000971', 'ENSG00000001084',
       'ENSG00000002587', 'ENSG00000002745', 'ENSG00000002834',
       'ENSG00000002933', 'ENSG00000003989', 'ENSG00000004468',
       'ENSG00000004809',
       ...
       'ENSG00000283654', 'ENSG00000283657', 'ENSG00000283663',
       'ENSG00000283674', 'ENSG00000283676', 'ENSG00000283683',
       'ENSG00000283684', 'ENSG00000283689', 'ENSG00000283693',
       'ENSG00000283696'],
      dtype='object', length=20765)

In [31]:
variance_threshold_selector(X_train, threshold=1)

Index(['ENSG00000000419', 'ENSG00000000457', 'ENSG00000000460',
       'ENSG00000000938', 'ENSG00000000971', 'ENSG00000001036',
       'ENSG00000001084', 'ENSG00000001167', 'ENSG00000001460',
       'ENSG00000001461',
       ...
       'ENSG00000283632', 'ENSG00000283633', 'ENSG00000283635',
       'ENSG00000283646', 'ENSG00000283652', 'ENSG00000283654',
       'ENSG00000283662', 'ENSG00000283667', 'ENSG00000283674',
       'ENSG00000283696'],
      dtype='object', length=23945)

In [35]:
count = 0
for i in range(len(lasso_selected_feature)):
    if lasso_selected_feature[i] in variance_selected_feature:
        count += 1
count

43

In [36]:
count = 0
for i in range(len(lasso_selected_feature)):
    if lasso_selected_feature[i] in ridge_selected_feature:
        count += 1
count

266

In [38]:
count = 0
for i in range(len(ridge_selected_feature)):
    if ridge_selected_feature[i] in variance_selected_feature:
        count += 1
count

7817

## Points to discuss
### 1) majority do not overlap in variance threshold selector
### 2) remove low-variance features, then apply different regularizers
### 3) heatmap to visualize selected genes
### 4) cross validation, I can use various performance metric to see if the method is rigorous. Would this be better than looking at whether same gene appears in 10 folds?

In [39]:
#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html